In [1]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify

In [2]:
app = Flask(__name__)

In [3]:
lemmatizer = WordNetLemmatizer()

with open('intents.json', encoding='utf-8') as file:
    intents = json.load(file)

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [5]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] =1;

    return np.array(bag)

In [6]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.10
    results = [[i,r] for i, r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list=[]
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list   

In [7]:
def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        return "Sorry, mujhe samajh nahi aaya. Kya aap dusre shabdon mein puchhna chahenge?"
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = i['responses']
            break
    return result

In [8]:
@app.route("/")
def home():
    return render_template("index.html")

In [9]:
@app.route("/get", methods=["POST"])
def chatbot_response():
    user_message = request.json['message']
    ints = predict_class(user_message)
    res = get_response(ints, intents)
    return jsonify({"response": res})

In [10]:
if __name__ == "__main__":
    mode = input("Enter 'api' for web mode, or 'cli' for console mode: ").lower()

    if mode == "cli":
        print("Console chatbot mode. Type 'exit' or 'quit' to stop.\n")
        while True:
            message = input("You: ")
            if message.lower() in ["exit", "quit", "stop", "band karo"]:
                print("Bot: Dhanyavaad! Aapka din shubh ho.")
                break
            ints = predict_class(message)
            res = get_response(ints, intents)
            print("Bot:", res)
        print("done")
    else:
        print("Starting API server...")
        app.run(debug=True)

Enter 'api' for web mode, or 'cli' for console mode:  cli


Console chatbot mode. Type 'exit' or 'quit' to stop.



You:  Hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Bot: ['Hello, mai aapki in vishayo me madad kar sakti hu: Admission, Eligiblity, Program Details, Hostel, Fees, Schlorship, Training, Location, Seats aadi. Aap kya janna chahenge?']


You:  B.Sc Nursing ka admission chahiye"


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Bot: ['B.Sc Nursing mein admission ke liye Biology avashyak hai.']


You:  Course ki details dijiye


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Bot: ['B.Sc Nursing program ek 4 saal ka full-time course hai, jisme hospital training bhi hoti hai.']


You:  exit


Bot: Dhanyavaad! Aapka din shubh ho.
done
